## Python Implementation of LSMC using Longstaff and Schwartz algorithm
Our implementation of the Least Squares Monte Carlo method for pricing american options is based on the work of Longstaff and Schwartz, presented in Longstaff and Schwartz (2001). 

American options call for a significant deviation from European or Asian-style pricing techniques, they allow for an early exercise before their maturity. This feature implies that when computing their value, we have to take into account the fact that a rational investor will continuously choose wether to exercise or hold the option, based on which choice will maximize her expected value. An American-style option pricing model must therefore compute the value of the option at every time step as the maximum between the payoff of exercising the option at that time step (early exercise value) and the expected value of holding the option to maturity (continuation value).



The first step in the Longstaff-Schwartz algorithm is to determine the expected payoff at maturity. This can be computed as the payoff of a conventional European option, as the continuation value is zero.
$$
E[\pi_T] = \max(E[S_{T}] - K,0)
$$


In [52]:
import Price_Models as pm
import pandas as pd
GMB_paths = pd.DataFrame(pm.BS_path(1, 0.1, 0.2, 1, 3, nPaths = 8))
GMB_paths = GMB_paths.rename(columns={0: 't = 0',  1: 't = 1',  2: 't = 2', 3: 't = 3'})
GMB_paths = GMB_paths.round(decimals = 2)
display(GMB_paths)

,t = 0,t = 1,t = 2,t = 3
0,1.0,1.11,1.33,1.36
1,1.0,0.89,0.80,0.71
2,1.0,1.01,0.77,0.82
3,1.0,0.99,0.98,1.01
4,1.0,0.96,0.98,1.15
5,1.0,1.15,1.19,1.46
6,1.0,1.20,1.10,0.97
7,1.0,1.03,1.18,1.29


In [54]:
import numpy as np
Option_payoff = pd.DataFrame(0, columns=GMB_paths.columns, index=GMB_paths.index)
Option_payoff['t = 3'] = np.maximum(1.1-GMB_paths['t = 3'],0)
Option_payoff = Option_payoff.round(decimals = 3)
display(Option_payoff)

,t = 0,t = 1,t = 2,t = 3
0,0,0,0,0.00
1,0,0,0,0.39
2,0,0,0,0.28
3,0,0,0,0.09
4,0,0,0,0.00
5,0,0,0,0.00
6,0,0,0,0.13
7,0,0,0,0.00


In [55]:
import numpy as np
import scipy.stats as stats

# Compute price of an american put option with the following parameters
S_0, K, T, sigma, r = 100,120,50/365,0.3,0.06
M = 1000

# generate the stock price paths
N = int(np.ceil(T*252)) #Time steps with trading days
S = pm.BS_path(S_0, r, sigma, T, N, nPaths = M)

payoff = np.maximum(K - S, 0)

# perform the least squares regression
dis_cfl = np.zeros((M, N+1)) # discounted cashflow at every timestep 
dis_cfl[:,N] = payoff[:,N] 
exercise_flag = np.zeros((M,N)) # when flag is 0 we should not exercise
cond = S[:,-1] < K # not in the money
exercise_flag[cond, -1] = 1
for i in range(N-1, 0, -1): # backward
    cond = S[:,i] < K
    X = np.column_stack([np.ones(M), S[:,i], S[:,i]**2])
    cond_x = X[cond, :]
    Y = np.exp(-r*dt) * dis_cfl[cond,i+1]
    beta = np.linalg.lstsq(cond_x, Y, rcond=None)[0]
    continue_val = np.dot(X, beta)
    continue_val[~cond] = 0
    cond_exercise = payoff[:,i] > continue_val
    exercise_flag[cond_exercise, i-1] = 1
    dis_cfl[:,i] = np.exp(-r*dt) * dis_cfl[:,i+1]
    dis_cfl[cond_exercise,i] = payoff[cond_exercise,i]

stopping_criteria = np.argmax(exercise_flag, axis=1) # first exercise point

actual_exercise = np.zeros_like(exercise_flag)
actual_exercise[np.arange(M), stopping_criteria] = exercise_flag[np.arange(M), stopping_criteria]
discount = (np.ones((M, N))*np.exp(-r*dt)).cumprod(axis=1)[::-1]
exp_payoff = (actual_exercise * payoff[:,1:] * discount).sum() / M
exp_payoff

20.28735500911359